In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)

In [4]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\tvhs1\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index = True, right_index = True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\tvhs1\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns = ['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=55,stratify = y)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])

hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 140


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (None, 160)               18720     
                                                                 
 dense_44 (Dense)            (None, 140)               22540     
                                                                 
 dense_45 (Dense)            (None, 1)                 141       
                                                                 
Total params: 41,401
Trainable params: 41,401
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "AlphabetSoupCharity_Optimization.hdf5"

In [44]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=5)

In [45]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5776 - accuracy: 0.7205
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5551 - accuracy: 0.7295
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5497 - accuracy: 0.7304
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5480 - accuracy: 0.7309
Epoch 5/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5468 - accuracy: 0.7332
Epoch 6/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5454 - accuracy: 0.7335
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5444 - accuracy: 0.7346
Epoch 8/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5437 - accuracy: 0.7356
Epoch 9/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5427 - accuracy: 0.7357
Epoch 10/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5423 - accuracy: 0.7355

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5703 - accuracy: 0.7294 - 500ms/epoch - 2ms/step
Loss: 0.570340633392334, Accuracy: 0.7294460535049438


In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])

hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 140
hidden_nodes_layer3 = 120


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 160)               18720     
                                                                 
 dense_47 (Dense)            (None, 140)               22540     
                                                                 
 dense_48 (Dense)            (None, 120)               16920     
                                                                 
 dense_49 (Dense)            (None, 1)                 121       
                                                                 
Total params: 58,301
Trainable params: 58,301
Non-trainable params: 0
_________________________________________________________________


In [48]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [49]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 5s 5ms/step - loss: 0.5684 - accuracy: 0.7222
Epoch 2/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5547 - accuracy: 0.7301
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5505 - accuracy: 0.7324
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5487 - accuracy: 0.7318
Epoch 5/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5468 - accuracy: 0.7329
Epoch 6/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5458 - accuracy: 0.7341
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5447 - accuracy: 0.7333
Epoch 8/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5442 - accuracy: 0.7358
Epoch 9/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5434 - accuracy: 0.7360
Epoch 10/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5422 - accuracy: 0.7364

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5537 - accuracy: 0.7289 - 550ms/epoch - 2ms/step
Loss: 0.553678035736084, Accuracy: 0.728863000869751


In [51]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])

hidden_nodes_layer1 = 160
hidden_nodes_layer2 = 140
hidden_nodes_layer3 = 120
hidden_nodes_layer4 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 160)               18720     
                                                                 
 dense_51 (Dense)            (None, 140)               22540     
                                                                 
 dense_52 (Dense)            (None, 120)               16920     
                                                                 
 dense_53 (Dense)            (None, 100)               12100     
                                                                 
 dense_54 (Dense)            (None, 1)                 101       
                                                                 
Total params: 70,381
Trainable params: 70,381
Non-trainable params: 0
_________________________________________________________________


In [52]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [53]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5677 - accuracy: 0.7203
Epoch 2/50
804/804 [==============================] - 5s 7ms/step - loss: 0.5558 - accuracy: 0.7293
Epoch 3/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5515 - accuracy: 0.7306
Epoch 4/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5495 - accuracy: 0.7324
Epoch 5/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5479 - accuracy: 0.7332
Epoch 6/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5477 - accuracy: 0.7317
Epoch 7/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5461 - accuracy: 0.7342
Epoch 8/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5456 - accuracy: 0.7341
Epoch 9/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5447 - accuracy: 0.7339
Epoch 10/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5440 - accuracy: 0.7348

In [54]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5574 - accuracy: 0.7297 - 530ms/epoch - 2ms/step
Loss: 0.5574058890342712, Accuracy: 0.72967928647995
